In [1]:
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using DifferentialEquations
using MeshIO

include("functions.jl")

display_transform (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Target Definition

In [ ]:
R = [1.0 0.0 0.0;   
     0.0 0.0 -1.0;
     0.0 1.0 0.0]

t = [0.04, -0.075, 0.32]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0   0.0   0.04
 0.0  0.0  -1.0  -0.075
 0.0  1.0   0.0   0.32
 0.0  0.0   0.0   1.0

### Hand Visualization

In [4]:
# Compile robot model
m = compile(robot)

kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)

#ADDITION OF A FRAME
display_frame(m, ls, "rh_fftip")
display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [5]:
#println("FRAMES")
#display(frames(robot))
#println("JOINTS")
#display(joints(robot))
#show(IOContext(stdout, :limit => false),  MIME("text/plain"), joints(robot))
#println("COORDINATES")
#show(coordinates(robot))
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(robot))
#println("COMPONENTS")
#display(components(robot))

Dict{String, VMRobotControl.CoordinateData} with 55 entries:
  "rh_lfdistal_mass_coord" => FramePoint{Float64, String}(rh_lfdistal, [0.0, 0.0, 0.012])
  "rh_rfdistal_mass_coord" => FramePoint{Float64, String}(rh_rfdistal, [0.0, 0.0, 0.012])
  "rh_thdistal_mass_coord" => FramePoint{Float64, String}(rh_thdistal, [0.0, 0.0, 0.01375])
  "rh_rftip_mass_coord" => FrameOrigin{String}(rh_rftip)
  "rh_palm_mass_coord" => FramePoint{Float64, String}(rh_palm, [0.0, 0.0, 0.035])
  "rh_rfknuckle_mass_coord" => FrameOrigin{String}(rh_rfknuckle)
  "rh_wrist_mass_coord" => FramePoint{Float64, String}(rh_wrist, [0.0, 0.0, 0.029])
  "rh_ffdistal_inertia_coord" => FrameAngularVelocity{String}("rh_ffdistal")
  "rh_thhub_mass_coord" => FrameOrigin{String}(rh_thhub)
  "rh_rfmiddle_mass_coord" => FramePoint{Float64, String}(rh_rfmiddle, [0.0, 0.0, 0.0125])
  "rh_mftip_mass_coord" => FrameOrigin{String}(rh_mftip)
  "rh_ffdistal_mass_coord" => FramePoint{Float64, String}(rh_ffdistal, [0.0, 0.0, 0.012])
  "rh_m

## Creating the Virtual Mechanism System

### Gravity Compensation and Joint Damping

In [5]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Virtual Mechanism Initialization

In [13]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)
target_rot = AxisAngle(SVector(1.0,0.0,0.0), Float64(π/2))
add_coordinate!(vms, FramePoint(".robot.$root", SVector(0.04, -0.075, 0.32));        id="Target position")
add_coordinate!(vms, QuaternionAttitude(".robot.rh_fftip",target_rot ) ; id = "ff orientation error")
add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", "Target position"); id= "ff position error")
add_coordinate!(vms, CoordDifference(".robot.rh_thtip_mass_coord", "Target position"); id = "th position error")



"th position error"

In [14]:
K = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
D = SMatrix{3, 3}(10., 0., 0., 0., 10.0, 0., 0., 0., 10.)

add_component!(vms, LinearSpring(K, "ff position error"); id="ff spring")
add_component!(vms, LinearDamper(D, "ff position error"); id="ff damper")

add_component!(vms, LinearSpring(K, "ff orientation error"); id = "angular spring")
add_component!(vms, LinearDamper(D, "ff orientation error"); id = "angular damper")

add_component!(vms, LinearSpring(K, "th position error"); id = "th spring")
add_component!(vms, LinearDamper(D, "th position error"); id = "th damper")

"th damper"

## Simulating the Robot

### Setting Up the Simulation

In [15]:
tspan = (0., 4.)
vms_compiled = compile(vms)
q = (zero_q(vms_compiled.robot), Float64[]) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), Float64[]) # Robot joint velocity, vm joint velocities
g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan)
@info "Simulating shadow robot with impedance control."

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-6, reltol=1e-6);

[ Info: Simulating shadow robot with impedance control.
ODE   0%|█                                              |  ETA: N/A
ODE   0%|█                                              |  ETA: 0:36:06
ODE   0%|█                                              |  ETA: 0:27:42
ODE   0%|█                                              |  ETA: 0:25:16
ODE   0%|█                                              |  ETA: 0:24:01
ODE   1%|█                                              |  ETA: 0:23:24
ODE   1%|█                                              |  ETA: 0:22:45
ODE   1%|█                                              |  ETA: 0:22:51
ODE   1%|█                                              |  ETA: 0:22:52
ODE   1%|█                                              |  ETA: 0:23:10
ODE   1%|█                                              |  ETA: 0:23:04
ODE   1%|█                                              |  ETA: 0:23:06
ODE   1%|█                                              |  ETA: 0:23:12
ODE   1%|█  

### Visualizing the Results

In [16]:
fig = Figure(size = (720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls, camera=:perspective, center=false)
cam.lookat[] = [0.25, 0.5, 0.1]
cam.eyeposition[] = [-0.4, -0.8, 0.5]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(robot)))
robotvisualize!(ls, plotting_kcache;)

savepath = joinpath(module_path, "docs/src/assets/shadowhand_orientation_grasp.mp4")
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);